# KAN-Image-IoT: Google Colab Setup

This notebook helps you set up and run the KAN-based person detection model in Google Colab with GPU acceleration.

## 1. Check GPU Availability

In [ ]:
import os
import sys
import torch

# Check for GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("No GPU available, using CPU")
    device = torch.device("cpu")

## 2. Clone Repository and Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/yourusername/kan-image-iot.git
%cd kan-image-iot

# Install dependencies
!pip install -q -r requirements.txt

## 3. Mount Google Drive for Saving Results

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create directory for experiment results
!mkdir -p /content/drive/MyDrive/kan-image-iot-results

# Create symbolic link to easily access results
!ln -sf /content/drive/MyDrive/kan-image-iot-results experiment_data

## 4. Prepare Dataset (Run Once)

In [ ]:
# Run this cell if you need to prepare the dataset
# It will download and process the VWW dataset
!python src/prepare_dataset.py

## 5. Train Model with GPU Acceleration

In [ ]:
# Train the model with GPU and mixed precision
!python src/train.py --device cuda --precision mixed --batch_size 128

## 6. Analyze Model Performance

In [ ]:
# Run model analysis
!python src/analyze.py --device cuda

## 7. Visualize Results

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import glob

# Find the latest experiment directory
exp_dirs = list(Path('experiment_data').glob('*'))
if exp_dirs:
    latest_exp = max(exp_dirs, key=os.path.getmtime)
    
    # Training curves
    train_plot = list(latest_exp.glob('figures/training_metrics.png'))
    if train_plot:
        plt.figure(figsize=(12, 8))
        plt.imshow(plt.imread(train_plot[0]))
        plt.axis('off')
        plt.title('Training Metrics')
        plt.show()
    
    # Confusion matrix
    cm_plot = list(latest_exp.glob('figures/confusion_matrix.png'))
    if cm_plot:
        plt.figure(figsize=(10, 8))
        plt.imshow(plt.imread(cm_plot[0]))
        plt.axis('off')
        plt.title('Confusion Matrix')
        plt.show()
else:
    print("No experiment results found. Run training first.")